# Calls

List the calls

In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

# Loads the .env file from the local path
load_dotenv()

address = 'mongodb://{user}:{password}@{host}:{port}'.format(
    user= os.getenv('MONGO_USER'),
    password= os.getenv('MONGO_PASSWORD'),
    host= os.getenv('MONGO_HOST'),
    port= os.getenv('MONGO_PORT')
)

client = MongoClient(address)
database=client["observertc-reports"]

# the collection we want to query
reportsDatabase = database.reports

# print('connected to %s' % address)

cursor = reportsDatabase.find( { "type": "CALL_EVENT", "payload.name": { "$in": ["CLIENT_JOINED", "CLIENT_LEFT"] } } )

def ticks_to_datetime(ticks):
    s, ms = divmod(ticks, 1000)
    return '{}.{:03d}'.format(time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(s)), ms)

rooms = {}
for record in cursor:
    callEvent = record["payload"]

    roomId = callEvent["roomId"]
    callId = callEvent["callId"]
    userId = callEvent['userId']
    clientId = callEvent['clientId']
    timestamp = callEvent['timestamp']

    calls = rooms.get(roomId, {})
    clients = calls.get(callId, {})
    client = clients.get(clientId, {
        "clientId": clientId,
        "userId": userId
    })
    
    if callEvent["name"] == "CLIENT_JOINED":
        client.update({ "joined": timestamp })
    else:
        client.update({ "left": timestamp })
    
    clients[clientId] = client
    calls[callId] = clients
    rooms[roomId] = calls

for room in rooms.items():
    roomId, calls = room
    print('\nRoom: %s\n--------' % roomId)
    for call in calls.items():
        callId, clients = call
        print('\nCall %s' % callId)
        for client in clients.values():
            try:
                userId, clientId, joined = client["userId"], client["clientId"], client["joined"]
                left = client.get("left", int(time.time() * 1000))
                duration = (left - joined) / (60 * 1000)
                print(  f"\n\tUserId: {userId}" +
                        f"\n\tClientId: {clientId}" +
                        f"\n\tJoined: {joined} {ticks_to_datetime(joined)}" + 
                        f"\n\tLeft: {left} {ticks_to_datetime(left)}" +
                        f"\n\tDuration: {duration} min")
            except Exception as e:
                #print(e)
                #print("Missing keys in call information", client)
                pass

## Max concurrent streams through SFU over time
This function expects there to be a unique Room id for each call

In [ ]:
# Match all tracks connected to an sfu
stage_match = {
    "$match": {
        "$or": [{ "type": "INBOUND_VIDEO_TRACK" }, { "type": "INBOUND_AUDIO_TRACK" },
                { "type": "OUTBOUND_VIDEO_TRACK" }, { "type": "OUTBOUND_AUDIO_TRACK" }],
        "$or": [{"payload.sfuStreamId": { "$exists" : True }}, 
                {"payload.sfuSinkId": { "$exists" : True }, "payload.sfuStreamId": { "$exists" : True }}]
   }
}
# group by clients
stage_group = {
   "$group": {
         "_id": {
             "clientId": "$payload.clientId",
             "sfuSinkId": "$payload.sfuSinkId",
             "sfuStreamId": "$payload.sfuStreamId"
          },
           "reportsNum": { "$sum": 1 }, 
   }
}

# the pipeline
pipeline = [
   stage_match, 
   stage_group,
]

cursor = reportsDatabase.aggregate(pipeline)
print("number of concurrent sfu streams", len(list(cursor)))